<a href="https://colab.research.google.com/github/kalendaracheal/Allowance-system/blob/master/shopliftingmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing Training Data

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/shoplifting/training')

label_types = os.listdir('/content/drive/MyDrive/shoplifting/training')
print (label_types)  

['shoplifting', 'notshoplifting']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rooms = []
for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/shoplifting/training' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/shoplifting/training' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


           tag                                         video_name
0  shoplifting  /content/drive/MyDrive/shoplifting/training/sh...
1  shoplifting  /content/drive/MyDrive/shoplifting/training/sh...
2  shoplifting  /content/drive/MyDrive/shoplifting/training/sh...
3  shoplifting  /content/drive/MyDrive/shoplifting/training/sh...
4  shoplifting  /content/drive/MyDrive/shoplifting/training/sh...
               tag                                         video_name
52  notshoplifting  /content/drive/MyDrive/shoplifting/training/no...
53  notshoplifting  /content/drive/MyDrive/shoplifting/training/no...
54  notshoplifting  /content/drive/MyDrive/shoplifting/training/no...
55  notshoplifting  /content/drive/MyDrive/shoplifting/training/no...
56  notshoplifting  /content/drive/MyDrive/shoplifting/training/no...


In [ ]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

# Preparing Test Data

In [ ]:
dataset_path = os.listdir('/content/drive/MyDrive/shoplifting/testing')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/shoplifting/testing')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/shoplifting/testing' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/shoplifting/testing' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['shoplifting', 'notshoplifting']
Types of activities found:  2
           tag                                         video_name
0  shoplifting  /content/drive/MyDrive/shoplifting/testing/sho...
1  shoplifting  /content/drive/MyDrive/shoplifting/testing/sho...
2  shoplifting  /content/drive/MyDrive/shoplifting/testing/sho...
3  shoplifting  /content/drive/MyDrive/shoplifting/testing/sho...
4  shoplifting  /content/drive/MyDrive/shoplifting/testing/sho...
               tag                                         video_name
16  notshoplifting  /content/drive/MyDrive/shoplifting/testing/not...
17  notshoplifting  /content/drive/MyDrive/shoplifting/testing/not...
18  notshoplifting  /content/drive/MyDrive/shoplifting/testing/not...
19  notshoplifting  /content/drive/MyDrive/shoplifting/testing/not...
20  notshoplifting  /content/drive/MyDrive/shoplifting/testing/not...


In [ ]:
!pip install git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-6dcc8o4e
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-6dcc8o4e
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=184182 sha256=6ebc036793888f229e8fcaed21c70e8e22426b583ed88b740e66a439efe1b129
  Stored in directory: /tmp/pip-ephem-wheel-cache-n20aok6t/wheels/cc/c4/d8/5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Successfully built tensorflow-docs


In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data preparation

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 57
Total videos for testing: 21


,Unnamed: 0,video_name,tag
18,18,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
26,26,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
41,41,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
3,3,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
9,9,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
49,49,/content/drive/MyDrive/shoplifting/training/no...,notshoplifting
1,1,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
32,32,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
44,44,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting
24,24,/content/drive/MyDrive/shoplifting/training/sh...,shoplifting


# Feed the videos to a network:

In [ ]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


# Feature Extraction

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87924736/87910968 [==============================] - 0s 0us/step


Label Encoding
# StringLookup layer encode the class labels as integers.

In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['notshoplifting', 'shoplifting']


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

# Finally, we can put all the pieces together to create our data processing utility.**bold text**

In [ ]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters


Frame features in train set: (57, 20, 2048)
Frame masks in train set: (57, 20)
train_labels in train set: (57, 1)
test_labels in train set: (21, 1)


## The sequence model
# Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)

    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
1/2 [==============>...............] - ETA: 8s - loss: 0.6167 - accuracy: 0.7188
Epoch 1: val_loss improved from inf to 0.73148, saving model to ./tmp/video_classifier
2/2 [==============================] - 11s 2s/step - loss: 0.5794 - accuracy: 0.7692 - val_loss: 0.7315 - val_accuracy: 0.5556
Epoch 2/30
1/2 [==============>...............] - ETA: 0s - loss: 0.4416 - accuracy: 0.9062
Epoch 2: val_loss did not improve from 0.73148
2/2 [==============================] - 0s 45ms/step - loss: 0.4278 - accuracy: 0.9231 - val_loss: 0.7530 - val_accuracy: 0.5556
Epoch 3/30
1/2 [==============>...............] - ETA: 0s - loss: 0.3764 - accuracy: 0.9688
Epoch 3: val_loss did not improve from 0.73148
2/2 [==============================] - 0s 39ms/step - loss: 0.3822 - accuracy: 0.9744 - val_loss: 0.7614 - val_accuracy: 0.5556
Epoch 4/30
1/2 [==============>...............] - ETA: 0s - loss: 0.3854 - accuracy: 0.9688
Epoch 4: val_loss did not improve from 0.73148
2/2 [================

# **Inference**

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


In [ ]:
## for short videos
from IPython.display import HTML
from base64 import b64encode
## for large videos
!pip install -U kora
from kora.drive import upload_public
url = upload_public('/content/drive/MyDrive/shoplifting/testing/shoplifting/Shoplift15.mp4')
# then display it
from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 5.7 MB/s 
     |████████████████████████████████| 1.6 MB 54.9 MB/s 


FileNotFoundError: ignored